In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from pathlib import Path
from fastai.basic_train import Learner
from ner_data import DEV, TEST, TRAIN, get_data_bunch
from optimizer import BertAdam
from learner import BertForNER, ner_loss

## Get Data

download the conll-2003 dataset
for example from https://github.com/Nidhi-K/Natural-Language-Processing-Projects/tree/master/Sequential%20CRF%20for%20NER/data
</br>(note: english test set is blind)
Full english dataset: https://github.com/kyzhouhzau/BERT-NER/tree/master/NERdata

and save it to DATA_PATH

In [5]:
DATA_PATH = Path('./data/conll-2003/eng')
DATA_PATH.mkdir(parents=True, exist_ok=True)

CSV_PATH = Path('./data/conll-2003/csv')
CSV_PATH.mkdir(parents=True, exist_ok=True)

DATA_BUNCH_PATH = Path('./data/conll-2003/data_bunch')
DATA_BUNCH_PATH.mkdir(parents=True, exist_ok=True)

ENG = {
    TRAIN: 'train.txt',
    DEV: 'dev.txt',
    TEST: 'test.txt'
}


In [6]:
# ! svn co 'https://github.com/Nidhi-K/Natural-Language-Processing-Projects/trunk/Sequential CRF for NER/data/' {DATA_PATH}
# ! svn co 'https://github.com/kyzhouhzau/BERT-NER/trunk/NERdata/' {DATA_PATH} 

In [7]:
# x = [DATA_PATH/file for file in [TRAIN, DEV, TEST]]
# conll_to_csv(CSV_PATH,x)

In [18]:
data = get_data_bunch(DATA_BUNCH_PATH, ENG, batch_size=1)

  0%|          | 0/10 [00:00<?, ?it/s]

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, 'O': 3, 'B-PER': 4, 'I-PER': 5, 'B-ORG': 6, 'I-ORG': 7, 'B-LOC': 8, 'I-LOC': 9, 'B-MISC': 10, 'I-MISC': 11}
{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, 'O': 3, 'B-PER': 4, 'I-PER': 5, 'B-ORG': 6, 'I-ORG': 7, 'B-LOC': 8, 'I-LOC': 9, 'B-MISC': 10, 'I-MISC': 11}


100%|██████████| 10/10 [00:00<00:00, 1905.46it/s]

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, 'O': 3, 'B-PER': 4, 'I-PER': 5, 'B-ORG': 6, 'I-ORG': 7, 'B-LOC': 8, 'I-LOC': 9, 'B-MISC': 10, 'I-MISC': 11}
50
10
10


In [19]:
data.batch_size

1

In [20]:
model = BertForNER.from_pretrained('bert-base-uncased', num_labels=12)

In [21]:
learn = Learner(data, model, BertAdam, loss_func=ner_loss)

In [22]:
learn.lr_find()

tensor(1214.8503, grad_fn=<SumBackward0>)
tensor(1211.9034, grad_fn=<SumBackward0>)
tensor(1218.9008, grad_fn=<SumBackward0>)
tensor(1194.9626, grad_fn=<SumBackward0>)
tensor(1202.4504, grad_fn=<SumBackward0>)
tensor(1214.4149, grad_fn=<SumBackward0>)
tensor(1176.1028, grad_fn=<SumBackward0>)
tensor(1223.6204, grad_fn=<SumBackward0>)
tensor(1142.3104, grad_fn=<SumBackward0>)
tensor(1177.3257, grad_fn=<SumBackward0>)
tensor(1127.7219, grad_fn=<SumBackward0>)
tensor(1112.7767, grad_fn=<SumBackward0>)
tensor(1099.4880, grad_fn=<SumBackward0>)
tensor(1038.4785, grad_fn=<SumBackward0>)
tensor(1072.7346, grad_fn=<SumBackward0>)
tensor(1021.4463, grad_fn=<SumBackward0>)
tensor(939.7308, grad_fn=<SumBackward0>)
tensor(984.4964, grad_fn=<SumBackward0>)
tensor(815.6038, grad_fn=<SumBackward0>)
tensor(781.8853, grad_fn=<SumBackward0>)
tensor(735.1388, grad_fn=<SumBackward0>)
tensor(562.3269, grad_fn=<SumBackward0>)
tensor(419.0259, grad_fn=<SumBackward0>)
tensor(230.8863, grad_fn=<SumBackward0>)


In [ ]:
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit(1,0.003)